In [18]:
import pandas as pd

# Load the diabetic patient's data
patient_data = pd.read_csv('patient_data.csv')

# Load the sugar data
sugar_data = pd.read_csv('sugar_data.csv')

# Load the meal data
meal_data = pd.read_csv('meal_data.csv')




In [19]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_openai import ChatOpenAI
import os
from typing import Literal
from langgraph.graph import Graph

os.environ['OPENAI_API_KEY'] = "sk-proj-S0PZemc2JGWt4B00fwANT3BlbkFJMI1toxyxHjIYAUkD6UlR"
os.environ['NVIDIA_API_KEY'] = "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"

model = ChatNVIDIA(model="meta/llama3-70b-instruct", api_key="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")


AgentState = {}
AgentState["messages"] = []
AgentState["grocerylist"] = ""
AgentState["mealplan"] = ""

In [20]:
def patient_goal(input):
    messages = input["messages"]
    user_input = messages[-1]
    print(user_input) 
    query = user_input
    response = model.invoke(query)
    input["messages"].append(response.content)
    return input

def router(input):
    messages = input["messages"]
    user_input = messages[-1]
    if "diabetes" in user_input.lower():
        return "diabetes"
    elif "weightloss" in user_input.lower() or "weight loss" in user_input.lower():
        return "weightloss"
    else:
       return "unknown"
   

def diabetes_mealplan(input):
    messages = input["messages"]
    user_input = messages[-1]
    query = "Based on this response: \n" + user_input + prompt + "\n Please provide a detailed specific analysis and recommendations for managing the diabetic patient's condition, considering their sugar levels, dietary requirements, and any other relevant factors. Your response should be comprehensive and tailored to the patient's specific needs. Based on the correlation between blood sugar level and meal data can you create a weekly mealplan recommendation. Also make this based on cuisine type specified in Patient data as preffered Cuisine"
    response = model.invoke(query)
    input["mealplan"] = response.content
    return input

def weightloss_mealplan(input):
    messages = input["messages"]
    user_input = messages[-1]
    query = "Based on this response: \n" + user_input + prompt + "\n Please provide a detailed specific analysis and recommendations for managing patient's weightloss considering dietary requirements, and any other relevant factors such as BMI. Your response should be comprehensive and tailored to the patient's specific needs. Based on the correlation between patient data, BMI and meal data can you create a weekly mealplan recommendation. Also make this based on cuisine type the patient follows"
    response = model.invoke(query)
    input["mealplan"] = response.content
    return input

def grocery_list_generator(input):
    mealplan = input["mealplan"]
    query = "Based on this response: \n" + mealplan + "\n Based on the weekly meal plan information create a weekly shopping list of groceries to buy. Include quantity as well."
    response = model.invoke(query)
    input["grocerylist"] = response.content
    return input


In [21]:
from langgraph.graph import Graph
#def generate_response(input_text):

 # Create a new graph
workflow = Graph()  

workflow.add_node("Patient_goal", patient_goal)
workflow.add_node("diabetes_mealplan", diabetes_mealplan)
workflow.add_node("weightloss_mealplan", weightloss_mealplan)
workflow.add_node("Grocery_list", grocery_list_generator)
workflow.add_conditional_edges(
  "Patient_goal",
    router,
        {
            "diabetes": "diabetes_mealplan",
            "weightloss": "weightloss_mealplan",
            "unknown": "__end__"
        },
    )
    
workflow.add_edge("diabetes_mealplan", "Grocery_list")
workflow.add_edge("weightloss_mealplan", "Grocery_list")

workflow.add_edge("Grocery_list", "__end__")

workflow.set_entry_point("Patient_goal")

app = workflow.compile()
#inputs = {"messages": [input_text]}
#answer = app.invoke(inputs)


NameError: name 'inputs' is not defined

In [ ]:

print(app.get_graph().draw_mermaid())

In [ ]:
from langchain_core.runnables.graph import CurveStyle, NodeColors, MermaidDrawMethod
from IPython.display import display, HTML, Image

display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)


NameError: name 'app' is not defined

In [ ]:
import pandas as pd

# Load the diabetic patient's data
patient_data = pd.read_csv('patient_data.csv')
sugar_data = pd.read_csv('sugar_data.csv')
meal_data = pd.read_csv('meal_data.csv')
goal = "diabetes"

# Convert dataframes to string
patient_data_str = patient_data.to_string(index=False)
sugar_data_str = sugar_data.to_string(index=False)
meal_data_str = meal_data.to_string(index=False)

# Create the prompt
prompt = f"""
Based on the following patient information:

Patient Data:
{patient_data_str}

Sugar Data:
{sugar_data_str}

Meal Data:
{meal_data_str}

 Based on the patient goal provided identify whether patient goal is to manage diabetes or manage weightloss. Give output as diabetes or weightloss or unknown

 """

print(prompt)



Based on the following patient information:

Patient Data:
 Patient ID     Name  Age Gender  Weight (kg)  Height (cm) Diabetes Type Preferred Cuisine
          1 John Doe   45   Male           85          175        Type 2            Kerala

Sugar Data:
      Date  Time  Blood Sugar Level (mg/dL)
2024-06-01 08:00                        110
2024-06-01 12:00                        150
2024-06-01 18:00                        140
2024-06-02 08:00                        105
2024-06-02 12:00                        145
2024-06-02 18:00                        135
2024-06-03 08:00                        108
2024-06-03 12:00                        148
2024-06-03 18:00                        130

Meal Data:
      Date      Meal                          Food  Quantity  Sugar Intake (g)
2024-06-01 Breakfast                       Oatmeal     1 cup                 1
2024-06-01 Breakfast                  Orange Juice     1 cup                21
2024-06-01 Breakfast                Scrambled Eggs      

In [ ]:
print(answer["messages"][-1])

**Analysis and Recommendations**

Based on the provided data, here is a detailed analysis and recommendations for managing John Doe's diabetes:

**Blood Sugar Levels:**

* John's average blood sugar level is 129.5 mg/dL, which is slightly above the normal range.
* His blood sugar levels are highest in the afternoon, around 12:00 pm, and lowest in the morning, around 08:00 am.
* There is a noticeable spike in blood sugar levels after meals, particularly after breakfast and lunch.

**Dietary Analysis:**

* John's diet is high in sugar intake, with an average daily sugar consumption of 63.3 grams.
* The majority of his sugar intake comes from breakfast foods, such as orange juice, Greek yogurt, banana, pancakes with syrup, and apple.
* His lunch and dinner options are relatively low in sugar, but still contribute to his overall sugar intake.

**Correlation between Blood Sugar Level and Meal Data:**

* There is a strong correlation between John's blood sugar levels and his sugar intake. Sp

In [ ]:
print(answer["grocerylist"])

Based on the daily meal plan recommendation, here is a weekly shopping list of groceries to buy:

**Grains:**

* Brown rice (2 bags, 2 lbs each)
* Idli rice (1 bag, 2 lbs)

**Proteins:**

* Fish (1 bunch, 1 lb)
* Chicken breast (1 pack, 1 lb)
* Chickpeas (1 can, 15 oz)
* Eggs (1 dozen)

**Vegetables:**

* Mixed vegetables (broccoli, carrots, green beans) (2 bunches, 2 lbs each)
* Green beans (1 bunch, 1 lb)
* Cauliflower (1 head, 1 lb)
* Fresh berries (1 container, 1 pint)
* Citrus fruits (oranges, lemons) (4-5 fruits)

**Dairy/Alternatives:**

* Coconut milk (2 cans, 14 oz each)
* Greek yogurt (2 cups, 16 oz each)
* Almond milk or other low-sugar milk alternative (1 gal, 128 oz)

**Pantry:**

* Coconut oil (1 jar, 14 oz)
* Spices (e.g., coriander, cumin, turmeric) (1 pack)
* Chutney ingredients (coconut, coriander, green chilies) (1 pack)
* Sambar powder (1 pack)
* Thoran mix (1 pack)
* Salt (1 bag, 1 lb)
* Pepper (1 bag, 1 lb)

**Snacks:**

* Nuts and seeds (e.g., almonds, pumpkin se